# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine

import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC

from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /home/jens/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jens/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jens/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jens/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_database.db')
df = pd.read_sql('messages', engine)
df.head()
X = df['message']
Y = df.drop(labels=['genre', 'id', 'message', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    #replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) 
    clean_tokens = []
 
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    filtered_tokens = [w for w in clean_tokens if not w in stop_words] 
        
    return filtered_tokens

for i in range(10):
    print(tokenize(df['message'][i]))
    print(df['message'][i])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Weather update - a cold front from Cuba that could pass over Haiti
['hurricane']
Is the Hurricane over or is it not over
['looking', 'someone', 'name']
Looking for someone but no name
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
says: west side of Haiti, rest of the country today and tonight
['information', 'national', 'palace']
Information about the National Palace-
['storm', 'sacred', 'heart', 'jesus']
Storm at sacred heart of jesus
['please', 'need', 'tent', 'water', 'silo', 'thank']
Please, we need tents and water. We are in Silo, Thank you!
['would', 'like', 'receive', 'message', 'thank']
I would like to receive the messages, thank you
['croix', 'de', 'bouqu

In [4]:
tokenize("that's a test")

['thats', 'test']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])



In [6]:
print(pipeline)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8d83c08620>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
parameters = {
    'clf__estimator__n_estimators': 10,
    'clf__estimator__criterion': 'entropy',
    'clf__estimator__max_features': ('auto'),#, 'log2'),
    'clf__estimator__min_samples_split': 2,
    'vect__ngram_range':  (1, 2),
    'tfidf__use_idf': False,
    'vect__max_df': 1.
}

pipeline.set_params(**parameters)

pipeline.fit(X_train, Y_train)
#Y_pred = pipeline.predict(X_test)


In [22]:
Y_pred = cv.best_estimator_.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
print(cv.best_estimator_.score(X_test, Y_test))

0.6737967914438503


In [28]:
Y_test_array = Y_test.to_numpy().T

In [33]:
Y_test.sum()

related                   4993
request                   1134
offer                       24
aid_related               2689
medical_help               498
medical_products           349
search_and_rescue          186
security                   111
military                   201
water                      432
food                       772
shelter                    583
clothing                   111
money                      165
missing_people              74
refugees                   227
death                      273
other_aid                  863
infrastructure_related     449
transport                  253
buildings                  328
electricity                140
tools                       36
hospitals                   79
shops                       30
aid_centers                 76
other_infrastructure       306
weather_related           1795
floods                     494
storm                      614
fire                        73
earthquake                 626
cold    

In [29]:
roc_score = []
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    roc = roc_auc_score(Y_test_array[column], Y_pred.T[column])
    print('roc auc score', roc)
    roc_score.append(roc)
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

       False       0.92      0.84      0.88      1552
        True       0.95      0.98      0.96      4993

    accuracy                           0.94      6545
   macro avg       0.94      0.91      0.92      6545
weighted avg       0.94      0.94      0.94      6545

roc auc score 0.9083432011000969
[[1302  250]
 [ 111 4882]]
request
              precision    recall  f1-score   support

       False       0.96      0.99      0.98      5411
        True       0.97      0.79      0.87      1134

    accuracy                           0.96      6545
   macro avg       0.96      0.89      0.92      6545
weighted avg       0.96      0.96      0.96      6545

roc auc score 0.8933562404886252
[[5383   28]
 [ 236  898]]
offer
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      6521
        True       1.00      0.42      0.59        24

    accuracy                           1

roc auc score 0.8788967404006516
[[5243   36]
 [ 298  968]]


In [30]:
print('average roc', np.mean(roc_score)) #0.83

average roc 0.8330064293471733


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
from pprint import pprint
pprint(pipeline.get_params())

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(criterion='entropy',
                                                       n_estimators=10)),
 'clf__estimator': RandomForestClassifier(criterion='entropy', n_estimators=10),
 'clf__estimator__bootstrap': True,
 'clf__estimator__ccp_alpha': 0.0,
 'clf__estimator__class_weight': None,
 'clf__estimator__criterion': 'entropy',
 'clf__estimator__max_depth': None,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__max_leaf_nodes': None,
 'clf__estimator__max_samples': None,
 'clf__estimator__min_impurity_decrease': 0.0,
 'clf__estimator__min_impurity_split': None,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__min_weight_fraction_leaf': 0.0,
 'clf__estimator__n_estimators': 10,
 'clf__estimator__n_jobs': None,
 'clf__estimator__oob_score': False,
 'clf__estimator__random_state': None,
 'clf__estimator__verbose': 0,
 'clf__estimator__warm_start': False,
 'clf__n_jobs': 

In [19]:
parameters = {
    'clf__estimator__n_estimators': [10],
    'clf__estimator__criterion': ('gini', 'entropy'),
    'clf__estimator__max_features': ('auto', 'sqrt'),#, 'log2'),
    'clf__estimator__min_samples_split': (2, 3),#,
    'vect__ngram_range': ((1, 1), (1, 2), (2,2)),
    'tfidf__use_idf': (True, False),
    #'vect__max_df': (0.1, 0.5, 1.0)
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=6, scoring='roc_auc_ovo', verbose=10)
grid_fit = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/jens/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
grid_fit.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_features': 'auto',
 'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 1)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle
outfile = open(MLmodel.pkl, 'wb')
pickle.dump(cv.best_estimator_, outfile, compress=1)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.